In [1]:
from DiSuQ.Torch import models
from scipy.optimize import minimize
from numpy import arange,linspace,meshgrid,array,log,argsort,pi,logspace,std,sqrt,log10,tile
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox,plotOptimization
from DiSuQ.Torch.optimization import domainParameters
from DiSuQ.Torch.components import indE,capE

In [ ]:
L0 = indE(.3e-9); L_ = indE(.3e-6); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(.5e-12); C_ = capE(1e-9) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(.5e-12); CJ_ = capE(1e-9) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 1000. ; J_ = .1 ; print('Junction Bound(GHz):',J_,J0+J_)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

In [ ]:
Lx = array([.8,24,128,512])
Cx = array([5e-4,.01])
Jx = array([200.])
CJx = array([5e-4,.01])

In [2]:
def representationCapacity(El,Ej,Ec,EcJ):
    basis = {'Chi':7,'Theta':64,'Phi':1024}
    # theta capacity is sufficient for the given parameter range
    basis = {'Chi':7,'Theta':12,'Phi':1024}
    flux0 = 24*pi*2
    return flux0,basis

In [ ]:
basis = {'Chi':7,'Theta':12,'Phi':30}
rep = 'R'; flux_point = ['Lx','Ly']
subspace = ['Lx','Cx','Jx','CJx']
circuit = models.zeroPi(basis,Ej=10.,Ec=5.,El=10.,EcJ=50.,sparse=False,
                        symmetry=True,_L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0),
                        ridge=True,flux0=24*pi)
initials = domainParameters([Lx,Cx,Jx,CJx],circuit,subspace)
len(initials)

In [ ]:
Bounds = []
for iD in keys:
    bound = components[iD].bounds()
    Bounds.append(bound)

In [ ]:
flux_range = linspace(0,.05,3,endpoint=True)
D0 = 1 ; Delta0 = 1

In [ ]:
def metric(parameters):
    El,Ec,Ej,EcJ = parameters
    basis,flux0 = representationCapacity(El,Ej,Ec,EcJ)
    zero_pi = models.SCzeroPi(basis,flux0,Ej=Ej,Ec=Ec,El=El,EcJ=EcJ)
    spectrum = zero_pi.get_spectrum_vs_paramvals('flux',flux_range,evals_count=3)
    G,I,II = spectrum.energy_table.T
    D = log10((II[0]-G[0])/(I[0]-G[0]))
    Delta = log10((I[0]-G[0])/std(I-G))
    return D,Delta

def objective(parameters):
    D,Delta = metric(parameters)
    return -D*D0 - Delta*Delta0

def logger(parameters):
    

In [ ]:
method='L-BFGS-B'; options=dict(ftol=0,maxiter=15)
for index,init in enumerate(initials):
    print(index,init)
    x0 = init['Lx'],init['Cx'],init['Jx'],init['CJx']
    results = minimize(objective,x0,method=method,options=options,jac=gradient,bounds=Bounds,callback=logger)

